# Test metrics on Toy Example

In [5]:
# FUNCTIONS / CLASSES :
import sys
%load_ext autoreload
%autoreload 2
sys.path.append("./code")
from metrics import *
from data_processing import *

metrics_dict = { "ngram_based" :[BLEU,
           Sacre_BLEU,
           ROUGE_1,
           ROUGE_L,
           ROUGE_S4,
           CHRF,
           CHRF_1,
           CHRF_pp,
           METEOR],
           
           "edit_based": [WER, TER],
           
           "embedding_based" : [BARY,
            DEPTH,
            MOVER]
}

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
reference = "Je me présente je m'appelle Henri"
good_candidate = "Je me présente je me prénomme Henri"
avg_candidate = "Bonjour mon nom est Henri"
bad_candidate = "J'aime les pizzas"

In [11]:
test_metrics(
    metrics = metrics_dict,
    reference = reference,
    good_candidate = good_candidate,
    avg_candidate = avg_candidate,
    bad_candidate = bad_candidate
)

METRIC TYPE :  NGRAM_BASED
METRIC NAME :  BLEU
good candidate :  0.7142857313156128
avg candidate :  0.16374613344669342
bad candidate :  0.0
TIME :  -0.00056295800004591
METRIC NAME :  Sacre_BLEU
good candidate :  0.7142857313156128
avg candidate :  0.16374613344669342
bad candidate :  0.0
TIME :  -1.9374999965293682e-05
METRIC NAME :  ROUGE_1
good candidate :  0.7058823704719543
avg candidate :  0.1538461595773697
bad candidate :  0.0
TIME :  0.0003142499999739812
METRIC NAME :  ROUGE_L
good candidate :  0.7058823704719543
avg candidate :  0.1538461595773697
bad candidate :  0.0
TIME :  0.00017112599994106859
METRIC NAME :  ROUGE_S4
good candidate :  0.5142857142857143
avg candidate :  0.0
bad candidate :  0.0
TIME :  -0.00015362599992840842
METRIC NAME :  CHRF
good candidate :  0.6306954117328964
avg candidate :  0.15168410023582476
bad candidate :  0.08836230975871608
TIME :  -0.00010387499992248195
METRIC NAME :  CHRF_1
good candidate :  0.6301317811012268
avg candidate :  0.15004

Depth Score Progress: 100%|██████████| 1/1 [00:00<00:00, 464.07it/s]


# Compute metrics for WMT22

In [ ]:
metrics_all = [metric for sublist in metrics_dict.values() for metric in sublist]
metrics_scores = compute_metrics(
    set_name = "WMT22",
    metrics = metrics_all,
    sample_size = 5000,
    path = "./data/metrics_scores_2.csv"
)